# RunnablePassthrough

`RunnablePassthrough` 는 데이터를 전달하는 역할을 합니다. 이 클래스는 `invoke()` 메서드를 통해 **입력된 데이터를 그대로 반환** 합니다.

이는 데이터를 변경하지 않고 파이프라인의 다음 단계로 전달하는 데 사용될 수 있습니다.

`RunnablePassthrough` 는 다음과 같은 시나리오에서 유용할 수 있습니다.

- 데이터를 변환하거나 수정할 필요가 없는 경우
- 파이프라인의 특정 단계를 건너뛰어야 하는 경우
- 디버깅 또는 테스트 목적으로 데이터 흐름을 모니터링해야 하는 경우

이 클래스는 `Runnable` 인터페이스를 구현하므로, 다른 `Runnable` 객체와 함께 파이프라인에서 사용될 수 있습니다.


In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH13-LCEL-Advanced")

## 데이터 전달하기

`RunnablePassthrough` 는 입력을 변경하지 않고 그대로 전달하거나 추가 키를 더하여 전달할 수 있습니다.

일반적으로 `RunnableParallel` 과 함께 사용되어 데이터를 맵의 새로운 키에 할당하는 데 활용됩니다.

`RunnablePassthrough()` 를 단독으로 호출하면 단순히 입력을 받아 그대로 전달합니다.

assign과 함께 호출된 RunnablePassthrough(`RunnablePassthrough.assign(...)`)는 입력을 받아 assign 함수에 전달된 추가 인자를 더합니다.


- `RunnableParallel` 클래스를 사용하여 **병렬로 실행 가능한 작업을 정의** 합니다.
- `passed` 속성에는 `RunnablePassthrough` 인스턴스를 할당하여 입력을 그대로 반환하도록 설정합니다.
- `extra` 속성에는 `RunnablePassthrough.assign()` 메서드를 사용하여 입력의 "num" 값에 3을 곱한 결과를 "mult" 키에 할당하는 작업을 정의합니다.
- `modified` 속성에는 람다 함수를 사용하여 입력의 "num" 값에 1을 더하는 작업을 정의합니다.
- `runnable.invoke()` 메서드를 호출하여 `{"num": 1}` 입력으로 병렬 작업을 실행합니다.


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    # 전달된 입력을 그대로 반환하는 Runnable을 설정합니다.
    passed=RunnablePassthrough(),
    # 입력의 "num" 값에 3을 곱한 결과를 반환하는 Runnable을 설정합니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 입력의 "num" 값에 1을 더한 결과를 반환하는 Runnable을 설정합니다.
    modified=lambda x: x["num"] + 1,
)

# {"num": 1}을 입력으로 Runnable을 실행합니다.
runnable.invoke({"num": 1})

In [ ]:
r = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)
r.invoke({"num": 1})

위의 예시에서 `passed` 키는 `RunnablePassthrough()`와 함께 호출되었으며, 이는 단순히 `{'num': 1}`을 전달합니다.

두 번째 줄에서는 숫자 값에 3을 곱하는 람다 함수와 함께 `RunnablePastshrough.assign`을 사용했습니다. 이 경우, `extra`는 원래 값에 `mult` 키가 추가된 `{'num': 1, 'mult': 3}`로 설정되었습니다.

마지막으로, `modified` 키를 사용하여 맵에 세 번째 키를 설정했는데, 이는 람다 함수를 사용하여 num에 1을 더한 단일 값을 설정하였으며, 그 결과 `modified` 키의 값은 `2`가 되었습니다.


## 검색기 예제

아래 예제에서는 `RunnablePassthrough` 사용하는 사용 사례를 살펴볼 수 있습니다.


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 텍스트로부터 FAISS 벡터 저장소를 생성합니다.
vectorstore = FAISS.from_texts(
    [
        "테디는 랭체인 주식회사에서 근무를 하였습니다.",
        "셜리는 테디와 같은 회사에서 근무하였습니다.",
        "테디의 직업은 개발자입니다.",
        "셜리의 직업은 디자이너입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)
# 벡터 저장소를 검색기로 사용합니다.
retriever = vectorstore.as_retriever()
# 템플릿을 정의합니다.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
# 템플릿으로부터 채팅 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")


# 문서를 포맷팅하는 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# 검색 체인을 구성합니다.
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("테디의 직업은 무엇입니까?")

In [ ]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("셜리의 직업은 무엇입니까?")